In [4]:
import json
import glob
import os
from collections import defaultdict

# 1) find all your seed files
patterns = ["MountainCarContinuous-v0_*.json", "Pendulum-v1_*.json"]
files = []
for p in patterns:
    files.extend(glob.glob(p))

# 2) bucket rewards by (env, episode_num)
data = defaultdict(lambda: defaultdict(list))
for fp in files:
    env, _ = os.path.basename(fp).rsplit('_', 1)
    with open(fp, 'r') as f:
        js = json.load(f)
    for ep in js['episodes']:
        data[env][ep['episode_num']].append(ep['reward'])

# 3) compute averages and dump one avg-file per env
for env, eps in data.items():
    out = {
        "experiment": {
            "policy": js["experiment"]["policy"],
            "environment": env,
            "seed": "avg",
            "start_time": js["experiment"]["start_time"]
        },
        "episodes": [],
        "evaluations": []
    }

    # sort by episode_num to keep order
    for ep_num in sorted(eps):
        rewards = eps[ep_num]
        avg_r = sum(rewards) / len(rewards)
        # preserve the same keys as your originals
        out["episodes"].append({
            "total_timesteps": js["episodes"][0]["total_timesteps"],
            "episode_num": ep_num,
            "episode_timesteps": js["episodes"][0]["episode_timesteps"],
            "reward": avg_r
        })

    fname = f"{env}_avg.json"
    with open(fname, 'w') as f:
        json.dump(out, f, indent=2)
    print(f"Wrote {fname}")


Wrote MountainCarContinuous-v0_avg.json
Wrote Pendulum-v1_avg.json
